## Cargamos las librerias

In [104]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import ast
import re
from textblob import TextBlob

## Cargamos el archivo json y pasamos los datos a un dataframe

In [105]:
ruta_reviews = "data2/australian_user_reviews.json"
filas_review = []
with open(ruta_reviews, encoding='utf-8') as f:
    for line in f.readlines():
        filas_review.append(ast.literal_eval(line))

# Transformamos a un dataframe
df_reviews = pd.DataFrame(filas_review)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [106]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [107]:
df_reviews.isnull().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

Podemos verificar que todos nuestros datos son de tipo objeto y que no tenemos ninguno que sea nulo

Verificamos si hay filas duplicadas en funcion del user_id y si son validas

In [108]:
duplicados = df_reviews[df_reviews.duplicated("user_id", keep=False)]
duplicados

,user_id,user_url,reviews
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,"[{'funny': '', 'posted': 'Posted June 16.', 'l..."
50,Rivtex,http://steamcommunity.com/id/Rivtex,"[{'funny': '', 'posted': 'Posted December 23, ..."
83,76561198094224872,http://steamcommunity.com/profiles/76561198094...,[]
119,DieMadchenschanderin,http://steamcommunity.com/id/DieMadchenschanderin,"[{'funny': '', 'posted': 'Posted August 29, 20..."
147,relesprit,http://steamcommunity.com/id/relesprit,"[{'funny': '', 'posted': 'Posted December 27, ..."
...,...,...,...
17819,76561198076474887,http://steamcommunity.com/profiles/76561198076...,"[{'funny': '', 'posted': 'Posted April 12.', '..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
18028,76561198075591109,http://steamcommunity.com/profiles/76561198075...,"[{'funny': '', 'posted': 'Posted December 26, ..."
18234,76561198092022514,http://steamcommunity.com/profiles/76561198092...,"[{'funny': '', 'posted': 'Posted July 3.', 'la..."


Podemos observar que tenemos 623 registros duplicados en cuanto a user_id pero puede ser que un mismo usuario haya realizado mas de un comentario por lo que se procede a analizar

In [109]:
user_id = "76561198156664158"
user_rows = duplicados.loc[duplicados["user_id"] == user_id, "reviews"]
for review_list in user_rows:
    for review in review_list:
        print(review)
    print("-" * 40)

{'funny': '', 'posted': 'Posted June 16.', 'last_edited': '', 'item_id': '252950', 'helpful': '0 of 1 people (0%) found this review helpful', 'recommend': True, 'review': 'love it'}
----------------------------------------
{'funny': '', 'posted': 'Posted June 16.', 'last_edited': '', 'item_id': '252950', 'helpful': '0 of 1 people (0%) found this review helpful', 'recommend': True, 'review': 'love it'}
----------------------------------------


Aqui se tomo un usuario para verificar si era el mismo comentario repetido y estabamos en lo correcto por lo que se procede a eliminar

In [110]:
df_reviews = df_reviews.drop_duplicates(subset="user_id", keep=False)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


En la variable reviews se observo que muchos de los registros se encuentran de forma anidada se analiza la estructura


In [111]:
df_reviews["reviews"][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

Corresponde desanidar la columna para luego poder proceder con el analisis. En este caso procedemos con la funcion "explode" para poder convertir cada elemento de la lista en una fila separada, replicando el índice y las demás columnas según sea necesario.

In [112]:
df_reviews_expanded = df_reviews.explode("reviews")
df_reviews_normalized = json_normalize(df_reviews_expanded["reviews"])
df_reviews_expanded.reset_index(drop=True, inplace=True)
df_reviews_normalized.reset_index(drop=True, inplace=True)
df_reviews_final = pd.concat([df_reviews_expanded,df_reviews_normalized], axis=1)
df_reviews_final = df_reviews_final.drop(columns='reviews')
df_reviews_final.head(2)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.


In [113]:
df_reviews_final.shape

(57595, 9)

In [114]:
df_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57595 entries, 0 to 57594
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      57595 non-null  object
 1   user_url     57595 non-null  object
 2   funny        57568 non-null  object
 3   posted       57568 non-null  object
 4   last_edited  57568 non-null  object
 5   item_id      57568 non-null  object
 6   helpful      57568 non-null  object
 7   recommend    57568 non-null  object
 8   review       57568 non-null  object
dtypes: object(9)
memory usage: 4.0+ MB


En el  caso de las nuevas columnas generadas observamos que no tenemos valores nulos pero que si hay registros que no tienen informacion por lo que se verifica

In [115]:
conteo_valores_vacios = df_reviews_final.apply(lambda col: col[col == ""].count())
conteo_valores_vacios

user_id            0
user_url           0
funny          49698
posted             0
last_edited    51632
item_id            0
helpful            0
recommend          0
review            30
dtype: int64

Solo tres columnas contienen valores vacios en el caso de la columna "funny" y "last_edited" no son necesarias para nuestro analisis por lo que se procede a eliminarlas y en cuanto a "reviews" se concluye que varios usuarios no dieron un comentario por lo que se coloca None. Tambien se observa que algunos registros de review contiene caracteres que pueden afectar el modelado posterior por lo que se corrigen

In [116]:
df_reviews_final = df_reviews_final.drop(columns=["funny","last_edited"])
df_reviews_final

,user_id,user_url,posted,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
57590,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 10.,70,No ratings yet,True,a must have classic from steam definitely wort...
57591,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 8.,362890,No ratings yet,True,this game is a perfect remake of the original ...
57592,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 3.,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
57593,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 20.,730,No ratings yet,True,:D


In [117]:
df_reviews_final = df_reviews_final.drop(columns=["user_url"])
df_reviews_final

,user_id,posted,item_id,helpful,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
57590,76561198312638244,Posted July 10.,70,No ratings yet,True,a must have classic from steam definitely wort...
57591,76561198312638244,Posted July 8.,362890,No ratings yet,True,this game is a perfect remake of the original ...
57592,LydiaMorley,Posted July 3.,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
57593,LydiaMorley,Posted July 20.,730,No ratings yet,True,:D


In [118]:
df_reviews_final.replace("",None, inplace=True)
df_reviews_final.head()

,user_id,posted,item_id,helpful,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [119]:
df_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57595 entries, 0 to 57594
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    57595 non-null  object
 1   posted     57568 non-null  object
 2   item_id    57568 non-null  object
 3   helpful    57568 non-null  object
 4   recommend  57568 non-null  object
 5   review     57538 non-null  object
dtypes: object(6)
memory usage: 2.6+ MB


In [120]:
review_nulos = df_reviews_final["review"].isnull().sum()
print(review_nulos)

57


In [121]:
nulos_largo = len(df_reviews_final["review"])
porcentaje_nulos_review = (review_nulos/nulos_largo)*100
porcentaje_nulos_review

0.09896692421217118

In [122]:
df_reviews_final=df_reviews_final.dropna(subset=["review"])
df_reviews_final = df_reviews_final.reset_index(drop=True)
df_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57538 entries, 0 to 57537
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    57538 non-null  object
 1   posted     57538 non-null  object
 2   item_id    57538 non-null  object
 3   helpful    57538 non-null  object
 4   recommend  57538 non-null  object
 5   review     57538 non-null  object
dtypes: object(6)
memory usage: 2.6+ MB


### Verificamos los registros que tenemos en review

In [127]:
reviews_col = df_reviews_final["review"].head(60)
reviews_col

0     Simple yet with great replayability In my opin...
1                    Its unique and worth a playthrough
2     Great atmosphere The gunplay can be a bit chun...
3     I know what you think when you see this title ...
4     For a simple its actually not all that simple ...
5     Very fun little game to play when your bored o...
6     A suitably punishing roguelike platformer  Win...
7         Run for fun What the hell kind of fun is that
8     Elegant integration of gameplay story world de...
9     Random drops and random quests with stat point...
10    Fun balance of tactics and strategy  Potential...
11    Fun world builder with plenty of option of how...
12    This game is so fun The fight sequences have b...
13    Really Really Really Great Game very good stor...
14    Just buy it already Great Story Great Multipla...
15    It was a great game from what I played right n...
16    The ending to this game is  Just buy it youll ...
17    Alan wake is a really good game the light 

In [128]:
def clean_and_show(text):
    cleaned_text = re.sub(r"[^a-zA-Z\s]", "", text)
    if cleaned_text != text:
        print(f"Texto original: {text}")
    return cleaned_text

df_reviews_final["review"] = df_reviews_final["review"].apply(clean_and_show)
# Esta funcion devuelve las filas que contienen caracteres especiales

### Verificamos que la funcion haya cumplido con el proposito

In [129]:
reviews_col = df_reviews_final["review"].head(60)
reviews_col

0     Simple yet with great replayability In my opin...
1                    Its unique and worth a playthrough
2     Great atmosphere The gunplay can be a bit chun...
3     I know what you think when you see this title ...
4     For a simple its actually not all that simple ...
5     Very fun little game to play when your bored o...
6     A suitably punishing roguelike platformer  Win...
7         Run for fun What the hell kind of fun is that
8     Elegant integration of gameplay story world de...
9     Random drops and random quests with stat point...
10    Fun balance of tactics and strategy  Potential...
11    Fun world builder with plenty of option of how...
12    This game is so fun The fight sequences have b...
13    Really Really Really Great Game very good stor...
14    Just buy it already Great Story Great Multipla...
15    It was a great game from what I played right n...
16    The ending to this game is  Just buy it youll ...
17    Alan wake is a really good game the light 

### De la variable "posted" solo necesitamos el año en el que se publico la review por lo que se procede a extraer para luego eliminarla

In [130]:
df_reviews_final["year_posted"] = df_reviews_final["posted"].str.extract(r'(\d{4})') 
df_reviews_final

,user_id,posted,item_id,helpful,recommend,review,year_posted
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability In my opin...,2011
1,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,Its unique and worth a playthrough,2011
2,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere The gunplay can be a bit chun...,2011
3,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014
4,js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple its actually not all that simple ...,2013
...,...,...,...,...,...,...,...
57533,76561198312638244,Posted July 10.,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN
57534,76561198312638244,Posted July 8.,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN
57535,LydiaMorley,Posted July 3.,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN
57536,LydiaMorley,Posted July 20.,730,No ratings yet,True,D,NaN


In [131]:
df_reviews_final.drop("posted" , axis = 1, inplace = True)
df_reviews_final

,user_id,item_id,helpful,recommend,review,year_posted
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability In my opin...,2011
1,76561197970982479,22200,No ratings yet,True,Its unique and worth a playthrough,2011
2,76561197970982479,43110,No ratings yet,True,Great atmosphere The gunplay can be a bit chun...,2011
3,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014
4,js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple its actually not all that simple ...,2013
...,...,...,...,...,...,...
57533,76561198312638244,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN
57534,76561198312638244,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN
57535,LydiaMorley,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN
57536,LydiaMorley,730,No ratings yet,True,D,NaN


In [132]:
nulos_year_posted = df_reviews_final.isnull().sum()
nulos_year_posted

user_id           0
item_id           0
helpful           0
recommend         0
review            0
year_posted    9749
dtype: int64

In [133]:
df_reviews_final["year_posted"].value_counts(normalize=True)

year_posted
2014    0.450899
2015    0.373140
2013    0.138693
2012    0.024838
2011    0.011049
2010    0.001381
Name: proportion, dtype: float64

Para no tener valores nulos vamos imputar aquellos registros nulos con la mediana dado que esta es menos sensible a valores extremos

In [134]:
df_reviews_final["year_posted"].fillna(2013, inplace=True)

In [135]:
df_reviews_final["year_posted"].isnull().sum()

0

Terminamos verificando si quedan valores nulos en alguna variable por revisar

In [136]:
df_reviews_final.isnull().sum()

user_id        0
item_id        0
helpful        0
recommend      0
review         0
year_posted    0
dtype: int64

### Variable "sentiment_analysis"

Crearemos la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna reemplazara la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos.

In [137]:
def analyze_sentiment(text):
    if not isinstance(text, str):
        return 1 
    if review is None:
        return 1
    blob = TextBlob(text)
    sentiment_polarity = blob.sentiment.polarity
    if sentiment_polarity < -0.1:
        return 0
    elif sentiment_polarity > 0.1:
        return 2
    else:
        return 1
    
df_reviews_final["sentiment_analysis"] = df_reviews_final["review"].apply(analyze_sentiment)

Analizamos el funcionamiento de la nueva funcion creada

In [98]:
for index, row in df_reviews_final.head(5).iterrows():
    review_text = row["review"]
    sentiment = row["sentiment_analysis"]
    print(f"Review {index}:")
    print(f"Texto del Review: {review_text}")
    print(f"Análisis de Sentimiento (0-Negativo, 1-Neutral, 2-Positivo): {sentiment}")
    print("\n")

Review 0:
Texto del Review: Simple yet with great replayability In my opinion does zombie hordes and team work better than left  dead plus has a global leveling system Alot of down to earth zombie splattering fun for the whole family Amazed this sort of FPS is so rare
Análisis de Sentimiento (0-Negativo, 1-Neutral, 2-Positivo): 2


Review 1:
Texto del Review: Its unique and worth a playthrough
Análisis de Sentimiento (0-Negativo, 1-Neutral, 2-Positivo): 2


Review 2:
Texto del Review: Great atmosphere The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequelso buy the game so I get a sequel
Análisis de Sentimiento (0-Negativo, 1-Neutral, 2-Positivo): 1


Review 3:
Texto del Review: I know what you think when you see this title Barbie Dreamhouse Party but do not be intimidated by its title this is easily one of my GOTYs You dont get any of that cliche game mechanics that all the latest games have this is simply good c

In [138]:
df_reviews_final.drop(columns=["review"], inplace=True)
df_reviews_final.head()

,user_id,item_id,helpful,recommend,year_posted,sentiment_analysis
0,76561197970982479,1250,No ratings yet,True,2011,2
1,76561197970982479,22200,No ratings yet,True,2011,2
2,76561197970982479,43110,No ratings yet,True,2011,1
3,js41637,251610,15 of 20 people (75%) found this review helpful,True,2014,2
4,js41637,227300,0 of 1 people (0%) found this review helpful,True,2013,1


In [100]:
df_reviews_final.isnull().sum()

user_id               0
user_url              0
item_id               0
helpful               0
recommend             0
year_posted           0
sentiment_analysis    0
dtype: int64

### Variable "helpful"

In [144]:
total_filas = df_reviews_final.shape[0]
no_reseña_helpful= df_reviews_final[df_reviews_final["helpful"]=="No ratings yet"]
num_rows_no_ratings = no_reseña_helpful.shape[0]
print(f"La cantidad de filas sin ratings es: {num_rows_no_ratings}")
porcentaje_no_ratings = (num_rows_no_ratings / total_filas) * 100
print(f"El porcentaje de filas sin ratings es: {porcentaje_no_ratings}")

La cantidad de filas sin ratings es: 29285
El porcentaje de filas sin ratings es: 50.89679863742222


Dado que la columna presenta la mitad de los registros sin ningun rating y ademas no es relevante para el posterior analisis y modelado, se procede a eliminarla

In [146]:
df_reviews_final.drop(columns=["helpful"], inplace=True)
df_reviews_final.head()

,user_id,item_id,recommend,year_posted,sentiment_analysis
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,1
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,1


In [147]:
df_reviews_final.shape

(57538, 5)

### Exportamos el CSV limpio 

In [148]:
archivo_limpio = "data2/user_reviews_limpio7.csv"
df_reviews_final.to_csv(archivo_limpio, index=False, encoding="utf-8")
print(f"Se guardó el archivo {archivo_limpio}")

Se guardó el archivo data2/user_reviews_limpio7.csv
